In [1]:
import numpy as np
import pandas as pd
from pandas_datareader import data as wb

In [2]:
tickers = ['AAPL','^GSPC']
data = pd.DataFrame()
for t in tickers:
    data[t] = wb.DataReader(t, data_source='yahoo', start='2010-1-1')['Adj Close']

In [3]:
sec_returns = np.log(data / data.shift(1))

In [4]:
cov = sec_returns.cov() *250
cov

,AAPL,^GSPC
AAPL,0.075991,0.031669
^GSPC,0.031669,0.030193


In [5]:
cov_with_market = cov.iloc[0,1]
cov_with_market

0.03166876077744593

In [6]:
market_var = sec_returns['^GSPC'].var()*250
market_var

0.030193010346167178

## Beta

##### $$ 
\beta_{pg} = \frac{\sigma_{pg,m}}{\sigma_{m}^2}
$$

In [7]:
PG_beta = cov_with_market / market_var
PG_beta

1.0488772207328472

In [8]:
def compute_beta(data, stock, market):
    log_returns = np.log(data / data.shift(1))
    cov = log_returns.cov()*250
    cov_w_market = cov.loc[stock,market]
    market_var = log_returns[market].var()*250
    return cov_w_market/market_var

In [9]:
compute_beta(data, "AAPL","^GSPC")

1.0488772207328472

## Expected Return: CAPM
##### $$
\overline{r_{pg}} = r_f + \beta_{pg}(\overline{r_{m}} - r_f) 
$$

In [10]:
def compute_capm(data, stock, market, riskfree = 0.025, riskpremium = 0.05):
    beta = compute_beta(data, stock, market)
    return (riskfree + (beta*riskpremium))

In [11]:
compute_capm(data, 'AAPL','^GSPC')

0.07744386103664236

## Sharpe Ratio
##### $$
Sharpe = \frac{\overline{r_{pg}} - r_f}{\sigma_{pg}}
$$

In [12]:
def compute_sharpe(data, stock, market, riskfree = 0.025, riskpremium=0.05):
    log_returns = np.log(data / data.shift(1))
    ret = compute_capm(data, stock, market, riskfree, riskpremium)
    return ((ret-riskfree)/(log_returns[stock].std()*250**0.5))

In [13]:
compute_sharpe(data, "AAPL","^GSPC")

0.19024568729137978